In [3]:
import com.fasterxml.jackson.databind.ObjectMapper
import com.fasterxml.jackson.dataformat.yaml.YAMLFactory
import com.fasterxml.jackson.dataformat.yaml.YAMLMapper
import com.fasterxml.jackson.dataformat.yaml.YAMLParser
import org.yaml.snakeyaml.LoaderOptions

val factory = YAMLFactory.builder()
    .loaderOptions(LoaderOptions().apply { codePointLimit = Int.MAX_VALUE })
    .build()
val mapper = ObjectMapper(factory)

In [4]:
import com.fasterxml.jackson.module.kotlin.readValue
import ds.types.raw.skyteam.Flight
import ds.types.raw.skyteam.SkyTeamFlights
import java.io.File

val data = mapper.readValue<Map<String, Map<String, Flight>>>(File("../data/SkyTeam-Exchange.yaml"))

In [5]:
import ds.utils.db.DB
import java.sql.Date

val unused = DB.getConnection().use { conn ->
    conn.prepareStatement("""
		INSERT INTO airlines.skyteam_exchange(user_code, user_fare, user_cls, flight_code, date, departure, arrival, status)
		VALUES (?, ?, ?, ?, ?, ?, ?, ?)
    """).use { stm ->
        data.forEach { date, flights ->
            flights.forEach { flightId, flight ->
                flight.frequentFlyers.forEach { flyerId, flyer ->
					stm.setString(1, flyerId)
                    stm.setString(2, flyer.fare)
                    stm.setString(3, flyer.cls)
                    stm.setString(4, flightId)
                    stm.setDate(5, Date.valueOf(date))
                    stm.setString(6, flight.from.uppercase())
                    stm.setString(7, flight.to.uppercase())
                    stm.setString(8, flight.status)
                    stm.addBatch()
                }
            }
        }
        stm.executeBatch()
    }
}

